# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [1]:
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

K. Antonelli  
O. Piastri  
G. Russel  
L. Stroll  
Red Bull Racing  
$100M  

What represents the "items" in the knapsack?  
The drivers and constructors.  

What represents the weights?  
Their costs.  

What represents the values?  
Their points.

In [19]:
# Load the data from the CSV files into a format that you are comfortable working with
# This can be arrays, dictionaries, or pd DataFrames

drivers_df = pd.read_csv('drivers.csv')
constructors_df = pd.read_csv('constructors.csv')

# Print/Display your data as well to confirm that you have successfully saved it.
print(drivers_df)
print(constructors_df)

                   Driver  Points  Cost
0            Lando Norris     100  29.6
1   Andrea Kimi Antonelli      61  19.3
2          George Russell      60  21.4
3          Max Verstappen      59  28.6
4           Oscar Piastri      55  23.0
5            Lance Stroll      33   9.3
6            Esteban Ocon      32   8.1
7          Alexaner Albon      28  12.8
8         Nico Hulkenberg      22   7.6
9          Oliver Bearman      22   6.7
10         Lewis Hamilton       3  23.6
11           Yuki Tsunoda       0  16.8
12            Liam Lawson       0   8.4
13        Charles LeClerc       0  25.3
14           Pierre Gasly      -7  10.6
15           Isack Hadjar      -9   5.0
16            Jack Doohan      -9   6.0
17           Carlos Sainz     -11  11.9
18      Gabriel Bortoleto     -11   4.8
19        Fernando Alonso     -36   7.6
       Constructor  Points  Cost
0          McLaren     172  30.6
1         Mercedes     136  23.3
2  Red Bull Racing      86  25.4
3             Haas      61  

# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem


$\text{maximize} \quad \sum_{i=1}^{n} x_i * p_i $  \
$\text{subject to} \quad \sum_{i=1}^{n} x_i * c_i \le 50M $ 


## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [20]:
budget = 50 # in millions

def constraints(max_cost, max_drivers, curr_cost, new_cost, num_drivers):
    # I don't know if this is what you guys were asking for.
    # This function returns True if the constraints are satisfied and False 
    # otherwise.

    return curr_cost+new_cost > max_cost or num_drivers+1 > max_drivers

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [24]:
def value_to_cost(items):
    items = items.copy()
    items["Ratio"] = items["Points"] / items["Cost"]
    return items

def sort_items(items): 
    items_sorted = items.sort_values(by="Ratio", ascending=False)
    return items_sorted

def greedy_driver_select(drivers_df):
    drivers = sort_items(value_to_cost(drivers_df))
    cost = 0
    num_drivers = 0
    # drivers_list = []
    drivers['Included'] = False

    for i, driver in drivers.iterrows():
        # if cost+driver["Cost"] > 50 or num_drivers+1 > 5:
        if constraints(50, 5, cost, driver["Cost"], num_drivers):
            continue
        else:
            cost += driver["Cost"]
            num_drivers += 1
            # drivers_list.append(driver["Driver"])
            drivers.at[i, 'Included'] = True
    return drivers


drivers_result = greedy_driver_select(drivers_df)
print(drivers_result)

                   Driver  Points  Cost     Ratio  Included
6            Esteban Ocon      32   8.1  3.950617      True
5            Lance Stroll      33   9.3  3.548387      True
0            Lando Norris     100  29.6  3.378378      True
9          Oliver Bearman      22   6.7  3.283582     False
1   Andrea Kimi Antonelli      61  19.3  3.160622     False
8         Nico Hulkenberg      22   7.6  2.894737     False
2          George Russell      60  21.4  2.803738     False
4           Oscar Piastri      55  23.0  2.391304     False
7          Alexaner Albon      28  12.8  2.187500     False
3          Max Verstappen      59  28.6  2.062937     False
10         Lewis Hamilton       3  23.6  0.127119     False
11           Yuki Tsunoda       0  16.8  0.000000     False
12            Liam Lawson       0   8.4  0.000000     False
13        Charles LeClerc       0  25.3  0.000000     False
14           Pierre Gasly      -7  10.6 -0.660377     False
17           Carlos Sainz     -11  11.9 

Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [ ]:
def compare(drivers, drivers_list, new_driver, total_points, cost, budget=50, max_drivers=5):
    best_swap = None
    best_score = total_points

    new_cost = new_driver['Cost']
    new_points = new_driver['Points']

    for j, (old_idx, _) in enumerate(drivers_list):
        old_driver = drivers.loc[old_idx]
        temp_cost = cost - old_driver['Cost'] + new_cost

        if temp_cost <= budget:
            temp_points = total_points - old_driver['Points'] + new_points
            if temp_points > best_score:
                best_score = temp_points
                best_swap = (j, old_idx, best_score)
    return best_swap
        

def redux_greedy_driver_select(drivers_df):
    drivers = sort_items(value_to_cost(drivers_df))

    cost = 0
    num_drivers = 0
    total_points = 0
    drivers_list = []

    for i, driver in drivers.iterrows():
        driver_cost = driver["Cost"]
        driver_points = driver["Points"]
        driver_name = driver["Driver"]

        if constraints(50, 5, cost, driver["Cost"], num_drivers):
            swap = compare(drivers, drivers_list, driver, total_points, cost)
            if swap:
                j, old_idx, new_score = swap
                cost = cost - drivers.at[old_idx, "Cost"] + driver_cost
                total_points = new_score
                drivers_list[j] = (i, driver_name)
        else:
            cost += driver_cost
            num_drivers += 1
            total_points += driver_points
            drivers_list.append(driver["Driver"])
    return drivers


print(redux_greedy_driver_select(drivers_df))

ValueError: too many values to unpack (expected 2)

In [17]:
def compare(drivers, drivers_list, new_driver, total_points, cost, budget=50, max_drivers=5):
    best_swap = None
    best_score = total_points

    new_cost = new_driver['Cost']
    new_points = new_driver['Points']

    for j, (old_idx, _) in enumerate(drivers_list):
        old_driver = drivers.loc[old_idx]
        temp_cost = cost - old_driver['Cost'] + new_cost

        if temp_cost <= budget:
            temp_points = total_points - old_driver['Points'] + new_points
            if temp_points > best_score:
                best_score = temp_points
                best_swap = (j, old_idx, best_score)

    return best_swap

def redux_greedy_driver_select(drivers_df):
    drivers = sort_items(value_to_cost(drivers_df.copy()))

    cost = 0
    num_drivers = 0
    total_points = 0
    drivers_list = []

    for i, driver in drivers.iterrows():
        driver_cost = driver["Cost"]
        driver_points = driver["Points"]
        driver_name = driver["Driver"]

        if constraints(50, 5, cost, driver["Cost"], num_drivers):
            swap = compare(drivers, drivers_list, driver, total_points, cost)
            if swap:
                j, old_idx, new_score = swap
                # Replace the old driver with the new one
                old_driver = drivers.loc[old_idx]
                cost = cost - old_driver["Cost"] + driver_cost
                total_points = new_score
                drivers.at[old_idx, 'Included'] = False
                drivers.at[i, 'Included'] = True
                drivers_list[j] = (i, driver_name)
        else:
            cost += driver_cost
            num_drivers += 1
            total_points += driver_points
            drivers.at[i, 'Included'] = True
            drivers_list.append((i, driver_name))

    return drivers_list, cost, num_drivers, total_points  # Or you can return the updated dataframe

# To see the results:
drivers_list, total_cost, num_drivers, total_points = redux_greedy_driver_select(drivers_df)
print(drivers_list)
print(f"Total cost: {total_cost}, Number of drivers: {num_drivers}, Points: {total_points}")


[(6, 'Esteban Ocon'), (5, 'Lance Stroll'), (0, 'Lando Norris')]
Total cost: 47.0, Number of drivers: 3, Points: 165


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 


$\text{maximize} \quad \sum_{}^{}$  \
$\text{subject to} \quad $ 


## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [ ]:
budget = 100 # in millions

# Replace with your code

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [5]:
def greedy_f1(drivers, constructors):
    # Replace with your code
    pass